In [1]:
#!conda install pandas
import pandas as pd
from IPython.display import display

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np

### Import dataset

In [3]:
orders = pd.read_csv("orders.csv")

In [4]:
products = pd.read_csv("products.csv")

In [5]:
products_quantity = pd.read_csv("orders_product_quantity.csv")

In [6]:
orders

,order_id,buyer_id,order_total
0,100001,72067,18.50
1,100194,68677,18.80
2,100476,37229,342.00
3,100708,20114,110.88
4,100988,45991,27.60
...,...,...,...
294,172448,25276,460.98
295,172839,35384,79.20
296,173312,56807,272.80
297,173400,35493,14.00


In [7]:
products

,product_id,product_name,category_name,product_price
0,126901,Fresh Ginger 500G,Fresh Fruits & Vegetables,3.00
1,130270,100 Plus Regular 24x325ml,Soft Drinks,35.46
2,137033,Strawberries 500g,Fresh Fruits & Vegetables,9.50
3,142331,Egg - Grade A 30pcs,"Dairy, Eggs, & Milk Substitutes",14.10
4,196831,Mushroom Fresh Straw 500g,Fresh Fruits & Vegetables,1.50
5,200808,Figo Chicken Slice Ham 500g,Frozen Food,7.92
6,247748,Potatoes,Fresh Fruits & Vegetables,2.20
7,256733,Spritzer Natural Mineral Mater 24x600ml,Mineral Water,24.80
8,141578,Evian Prestige Natural Mineral Water 24x330ml,Mineral Water,72.46


In [8]:
products_quantity

,id,order_id,product_id,product_quantity,sub_total
0,1,100001,196831,11,16.50
1,2,100194,247748,6,16.80
2,3,100476,130270,9,342.00
3,4,100708,200808,14,110.88
4,5,100988,126901,8,25.60
...,...,...,...,...,...
347,348,172448,130270,13,460.98
348,349,172839,200808,10,79.20
349,350,173312,256733,11,272.80
350,351,173400,196831,8,12.00


### Create data frame for recurring buyers and unique buyers

recurring buyers -> is the buyer who repeats the order / orders on several occasion

unique buyers-> is the buyer who only has ordered once/ dont repeat orders

In [9]:
select_order=orders.groupby("buyer_id").count()

In [10]:
select_order

,order_id,order_total
buyer_id,,
11285,5,5
11514,1,1
11573,1,1
11609,1,1
11636,1,1
...,...,...
72163,1,1
72190,1,1
72225,1,1


In [11]:
#select the buyer who only ordered more than one (but select only order_id column)
recurring=select_order.loc[select_order["order_id"]>1, ["order_id"]]

In [12]:
recurring

,order_id
buyer_id,
11285,5
11675,6
16728,7
17611,10
19513,5
19755,14
20462,6
25276,7
28039,5


In [13]:
#select the buyer who ordered only once (but select only order_id column)
unique=select_order.loc[select_order["order_id"]<2 ,["order_id"]]

In [14]:
unique

,order_id
buyer_id,
11514,1
11573,1
11609,1
11636,1
17604,1
...,...
72163,1
72190,1
72225,1


In [15]:
#calculate/sum the total order made by the each buyers
total_order=orders.groupby("buyer_id").sum()

In [16]:
#select only order_total for the dataframe
total_order=total_order.loc[:, ["order_total"]]

In [17]:
total_order

,order_total
buyer_id,
11285,214.50
11514,41.60
11573,99.20
11609,124.80
11636,104.00
...,...
72163,9.92
72190,57.00
72225,291.20


In [18]:
#merge the recurring buyers with total orders to know the sum of orders made by those buyers.
#merge by buyer_id
df_recurring = pd.merge(recurring, total_order, on=["buyer_id"])

In [19]:
df_recurring

,order_id,order_total
buyer_id,,
11285,5,214.50
11675,6,771.20
16728,7,2054.40
17611,10,2420.18
19513,5,617.50
19755,14,1630.00
20462,6,510.08
25276,7,823.38
28039,5,153.40


In [20]:
#merge the unique buyers with total orders to know the sum of orders made by those buyers.
#merge by buyer_id
df_unique = pd.merge(unique, total_order, on=["buyer_id"])

In [21]:
df_unique

,order_id,order_total
buyer_id,,
11514,1,41.60
11573,1,99.20
11609,1,124.80
11636,1,104.00
17604,1,20.00
...,...,...
72163,1,9.92
72190,1,57.00
72225,1,291.20


Now we get the dataframes for recurring buyers and unique buyers for further analysis.

### Data Analysis 

Finding the product which sold the most

In [22]:
#grouping the product_id
product_sales=products_quantity.groupby("product_id").sum()

In [23]:
#filter the dataframe. choose on product_quantity and subtotal columns only
product_sales=product_sales.loc[:, ["product_quantity" , "sub_total"]]

In [24]:
#sort the product quantity to find the ones which sold the most
product_sales.sort_values("product_quantity", ascending=False)

,product_quantity,sub_total
product_id,,
126901,790,2459.20
256733,380,8316.00
196831,335,502.50
130270,315,11716.00
142331,282,3727.90
137033,281,2669.50
200808,273,2162.16
247748,221,552.80


In [25]:
product_sales.count()

product_quantity    8
sub_total           8
dtype: int64

In [26]:
products.count()

product_id       9
product_name     9
category_name    9
product_price    9
dtype: int64

something isn't right, the total products is 9. but from the count of the products is 8. Is it because there is a product wasn't sold? let's check it

In [27]:
list1=product_sales.index.tolist()

In [28]:
list2=products["product_id"].tolist()

In [29]:
#find the product id which is not being sold
unsold_product=[item for item in list2 if item not in list1]

In [30]:
unsold_product

[141578]

In [31]:
#check whether the product has a record in sales
products_quantity.loc[products_quantity["product_id"]==unsold_product[0]]

,id,order_id,product_id,product_quantity,sub_total


In [32]:
#check which product is unsold
products.loc[products["product_id"]==unsold_product[0]]

,product_id,product_name,category_name,product_price
8,141578,Evian Prestige Natural Mineral Water 24x330ml,Mineral Water,72.46


#### calculate Gross Merchandise Value (GMV)

Gross Merchandise Value (GMV) is a metric that measures your total value of sales over a certain period of time

Formula:


Gross Merchandise Value =  Sales Price of Goods x Number of Goods Sold

In [33]:
products=products.loc[:, ["product_id" , "product_name", "product_price"]]

In [34]:
#merge the product details table and product sales table
df_products=pd.merge(products, product_sales, on=["product_id"]).sort_values("product_quantity", ascending=False)

In [35]:
df_products

,product_id,product_name,product_price,product_quantity,sub_total
0,126901,Fresh Ginger 500G,3.00,790,2459.20
7,256733,Spritzer Natural Mineral Mater 24x600ml,24.80,380,8316.00
4,196831,Mushroom Fresh Straw 500g,1.50,335,502.50
1,130270,100 Plus Regular 24x325ml,35.46,315,11716.00
3,142331,Egg - Grade A 30pcs,14.10,282,3727.90
2,137033,Strawberries 500g,9.50,281,2669.50
5,200808,Figo Chicken Slice Ham 500g,7.92,273,2162.16
6,247748,Potatoes,2.20,221,552.80


In [36]:
#create new column in the dataframe as GMV
df_products["GMV"]=df_products["product_price"]*df_products["product_quantity"]

In [37]:
df_products

,product_id,product_name,product_price,product_quantity,sub_total,GMV
0,126901,Fresh Ginger 500G,3.00,790,2459.20,2370.00
7,256733,Spritzer Natural Mineral Mater 24x600ml,24.80,380,8316.00,9424.00
4,196831,Mushroom Fresh Straw 500g,1.50,335,502.50,502.50
1,130270,100 Plus Regular 24x325ml,35.46,315,11716.00,11169.90
3,142331,Egg - Grade A 30pcs,14.10,282,3727.90,3976.20
2,137033,Strawberries 500g,9.50,281,2669.50,2669.50
5,200808,Figo Chicken Slice Ham 500g,7.92,273,2162.16,2162.16
6,247748,Potatoes,2.20,221,552.80,486.20


#### Calculate Gross Transaction Value (GTV)

GTV is the commission on the transaction price charged to the customer then multiplied by the number of items sold.

GTV = # of Transactions * AOV * % of Transaction

where:

* #of Transactions = the total number of transactions made and items sold through a marketplace, 
* AOV = the average order value,
* % of every transaction.

If suppliers sell 10,000 items across the entire marketplace for \\$100 each and a pay 10% fee for every transaction, the GTV will be \$100,000 (=100\*10000\*0.1). It's another effective way to calculate the gross revenue.

In [40]:
products_quantity

,id,order_id,product_id,product_quantity,sub_total
0,1,100001,196831,11,16.50
1,2,100194,247748,6,16.80
2,3,100476,130270,9,342.00
3,4,100708,200808,14,110.88
4,5,100988,126901,8,25.60
...,...,...,...,...,...
347,348,172448,130270,13,460.98
348,349,172839,200808,10,79.20
349,350,173312,256733,11,272.80
350,351,173400,196831,8,12.00


In [45]:
 product_price=products.loc[:, ["product_id","product_price"]]

In [48]:

product_revenue=pd.merge(products_quantity,product_price, on=["product_id"])
product_revenue

,id,order_id,product_id,product_quantity,sub_total,product_price
0,1,100001,196831,11,16.5,1.5
1,16,104012,196831,5,7.5,1.5
2,18,104666,196831,5,7.5,1.5
3,26,105808,196831,1,1.5,1.5
4,27,106275,196831,5,7.5,1.5
...,...,...,...,...,...,...
347,334,169764,142331,15,211.5,14.1
348,335,170022,142331,1,14.1,14.1
349,340,170472,142331,5,70.5,14.1
350,346,172324,142331,8,112.8,14.1


In [50]:
#calculate GTV for each order
#I assume we pay 4% fee for every transaction
#create a column for GTV to calculate for every order
product_revenue["GTV"]=product_revenue["product_quantity"]*product_revenue["product_price"]*0.04
product_revenue

,id,order_id,product_id,product_quantity,sub_total,product_price,GTV
0,1,100001,196831,11,16.5,1.5,0.660
1,16,104012,196831,5,7.5,1.5,0.300
2,18,104666,196831,5,7.5,1.5,0.300
3,26,105808,196831,1,1.5,1.5,0.060
4,27,106275,196831,5,7.5,1.5,0.300
...,...,...,...,...,...,...,...
347,334,169764,142331,15,211.5,14.1,8.460
348,335,170022,142331,1,14.1,14.1,0.564
349,340,170472,142331,5,70.5,14.1,2.820
350,346,172324,142331,8,112.8,14.1,4.512


In [54]:
#calculate the GTV for each product by group
#sum only for sub_total and GTV columns
product_revenue.groupby(["product_id"])[["product_quantity","sub_total", "GTV"]].sum().sort_values("GTV", ascending=False)

,product_quantity,sub_total,GTV
product_id,,,
130270,315,11716.00,446.7960
256733,380,8316.00,376.9600
142331,282,3727.90,159.0480
137033,281,2669.50,106.7800
126901,790,2459.20,94.8000
200808,273,2162.16,86.4864
196831,335,502.50,20.1000
247748,221,552.80,19.4480


product_id(130270) contributed for the most GTV

In [55]:
#the total GTV of the dataset
product_revenue["GTV"].sum()

1310.4184

In [58]:
df_recurring

,order_id,order_total
buyer_id,,
11285,5,214.50
11675,6,771.20
16728,7,2054.40
17611,10,2420.18
19513,5,617.50
19755,14,1630.00
20462,6,510.08
25276,7,823.38
28039,5,153.40


In [65]:
#calculate the average value order for the recurring order
df_recurring["order_each_transaction"]=df_recurring["order_total"]/df_recurring["order_id"]
df_recurring

,order_id,order_total,order_each_transaction
buyer_id,,,
11285,5,214.50,42.900000
11675,6,771.20,128.533333
16728,7,2054.40,293.485714
17611,10,2420.18,242.018000
19513,5,617.50,123.500000
19755,14,1630.00,116.428571
20462,6,510.08,85.013333
25276,7,823.38,117.625714
28039,5,153.40,30.680000


In [66]:
#calculate the average value order for the recurring order
df_recurring["order_each_transaction"].mean()

129.85390476190477

In [60]:
df_unique

,order_id,order_total
buyer_id,,
11514,1,41.60
11573,1,99.20
11609,1,124.80
11636,1,104.00
17604,1,20.00
...,...,...
72163,1,9.92
72190,1,57.00
72225,1,291.20


In [61]:
#calculate the average value order for the one time customer order
df_unique["order_total"].mean()

97.77624999999998

In [72]:
#the percentage recurring buyer vs one time buyer
recurring_num = df_recurring["order_id"].count()
unique_num = df_unique["order_id"].count()
print("The percentage of recurring buyer: ", round(recurring_num*100/(recurring_num+unique_num), 2))
print("The percentage of one time buyer: ", round(unique_num*100/(recurring_num+unique_num), 2))

The percentage of recurring buyer:  5.88
The percentage of one time buyer:  94.12


In [77]:
#calculate the ratio
print("The ratio of one time buyer to recurring buyer: ")
print(str(int(unique_num/recurring_num)) + ":1" )

The ratio of one time buyer to recurring buyer: 
16:1
